# To be run once to setup modules and paths


In [2]:
#Reload modules without shutting notebook
%load_ext autoreload
%autoreload 2

import os
import shutil
import glob
import numpy as np
import h5py
import pickle
from pandas import DataFrame
import gdal
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from PIL import Image, ImageDraw

# Do not show warnings
import warnings
warnings.filterwarnings('ignore')

# Upland burn modules
import utils.extract_aoi as extract
import utils.plot_data as plotd
from utils.get_configuration import get_configuration

# Define input variables if not calling externally
try:
    print("Input dataset from: {}".format(basefolder))     # check if defined
    polygon_file = os.path.join(datasets, polygon + ".geojson")

# if not defined raises an error and control shifts to except block. 
except:
    print ("Defining variables as not called externally") 
    
    # Get configuration information
    cstudy = "skye" 

    # Get configuration information
    basefolder, s1ardfolder, datasets, outfolder, tmpfolder, ofiles, hdfile, pfile, verbose, graphics = get_configuration(cstudy)
    
    # Burn Polygon
    polygon = "Skye_burn_extent_163"
    polygon_file = os.path.join(datasets, polygon + ".geojson")
    
# Output array
burn_subarray = None
non_burn_subarray = None
#print("Check: ",ofiles[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Defining variables as not called externally
Loaded configuration: {'sharedfolder': 'my_shared_data_folder'}.


# Display generated Skye subset

In [8]:
# Load pickle file and extract statistics
infile = open(pfile,'rb')
data_dict = pickle.load(infile)
infile.close()

# Convert to dataframe
df = DataFrame(data_dict, columns=['layer', 'sensor', 'date', 'stime', 'etime','polarisation', 'rorbit', 'direction'])
layers = len(df)
if verbose:
    print("Loading {} layers".format(layers))

In [3]:
# Read hdf file and display chosen band: VV or VH
if verbose:
    hf_object = h5py.File(hdfile, 'r')
    d = hf_object['s1ard']
    print("Displaying {} layers".format(layers))
    plotd.plot_images(d, df['date'].values, verb = verbose, hdf = 'VV')
    hf_object.close()

## Create subset for a polygon of interest

In [1]:
import geojson as gj
def create_subset(polygon_file, subarray, ofiles, hdfile):
    # Get coords for bounding box
    with open (polygon_file, 'r') as f:
        loadg = gj.loads(f.read())
    x, y = zip(*gj.utils.coords(loadg))
    min_x, max_x, min_y, max_y = min(x), max(x), min(y), max(y)
    if verbose:
        print("Input for GeoJSON ULeft {}:{} LRight {}:{} ".format(min_x, max_y, max_x, min_y))

    # Get coordinate information from GeoTIFF file
    if verbose:
        print("Loading {} into HDF file".format(ofiles[0]))
    ds = gdal.Open(ofiles[0], gdal.GA_ReadOnly)
    bands = ds.RasterCount
    image = ds.GetRasterBand(1).ReadAsArray()
    xdim, ydim = image.shape
    del image

    global pixelWidth
    global pixelHeight
    
    # Getting georeference info
    transform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    xOrigin = transform[0] # top left x 
    yOrigin = transform[3] # top left y 
    pixelWidth = transform[1] # w-e pixel resolution 
    pixelHeight = -transform[5] # n-s pixel resolution (negative value) 

    # Close HDF file
    ds = None

    # Computing [top left] [lower right]
    i1 = int((min_x - xOrigin) / pixelWidth)
    i2 = int((max_x - xOrigin) / pixelWidth) + 1 #we need to add 1 as int rounds down
    j1 = int((yOrigin - max_y) / pixelHeight)
    j2 = int((yOrigin - min_y) / pixelHeight) + 1 # we need to add 1 as int rounds down
    if verbose:
        print("Pixel coordinates for subset ULeft {}:{} LRight {}:{}".format(i1, j1, i2, j2))

    hf_object = h5py.File(hdfile, 'r')

    vect = []
    # first lets get the coordinates of the shape outline
    for x1, y1 in zip(x, y):
        xpix = ((x1 - xOrigin) / pixelWidth) - i1
        ypix = ((yOrigin - y1) / pixelHeight) - j1
        #vect.append([int(ypix), int(xpix)])
        vect.append(int(xpix))
        vect.append(int(ypix))
    img = Image.new('L',  (i2-i1,j2-j1), 0)                                                                                                                                                                                                                  
    draw = ImageDraw.Draw(img)                                                                                                                                                                                                                                        
    draw.polygon(vect, fill=1)
    mask = np.array(img)

    if verbose:
        print("Loading layers...")
    if subarray is None:
        # Using 4GB of chunk_cache_mem here ("rdcc_nbytes")
        with h5py.File(hdfile, 'r', rdcc_nbytes =1024**2*4000, rdcc_nslots=1e7) as h5f: 
            for layer in range(layers):
                print("Loading layer {} of {}".format(layer+1,layers), end='\r')
                if subarray is None:
                    subarray = h5f['s1ard'][layer, :, j1:j2, i1:i2] # only extract the layer and subset
                    #if np.mean(subarray) == 0:
                    #    subarray = None
                    #    df.drop(layer, inplace=True)
                    #    continue
                    subarray = np.expand_dims(subarray, axis=0)
                    continue
                else:
                    ind_arr = h5f['s1ard'][layer, :, j1:j2, i1:i2]
                    #if np.mean(ind_arr) == 0:
                    #    df.drop(layer, inplace=True)
                    #    continue
                    ind_arr = np.expand_dims(ind_arr, axis=0)
                subarray = np.vstack([subarray, ind_arr]) # stack them 
    else:
        print("Layers already loaded: ",subarray.shape)

    print("\n Extracting only polygon area...")
    mask =  np.broadcast_to(mask, subarray.shape) # make the mask the same shape
    #subarray[mask==0] = np.nan # set anything outside the polygon to np.nan

    # New upper-left X,Y values
    start_x = xOrigin + (i1 * pixelWidth)
    start_y = yOrigin - (j1 * pixelHeight)
    end_x = xOrigin + (i2 * pixelWidth)
    end_y = yOrigin - (j2 * pixelHeight)
    new_transform = (start_x, transform[1], transform[2], start_y, transform[4], transform[5])
    if verbose:
        print("Origin: Old-UL {:.3f} {:.3f} New-UL {:.3f} {:.3f} New-LR {:.3f} {:.3f}".
              format(xOrigin, yOrigin, start_x, start_y, end_x, end_y))
    return subarray, new_transform, mask[0,0,:,:]

In [14]:
import matplotlib.pyplot as plt 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
polar_dict = {'VV' : 0, 'VH' : 1}

list_of_dates = df['date'].values
infoW = widgets.Label("")

#subarray = None
burn_subarray, new_transform, mask = create_subset(polygon_file, burn_subarray, ofiles, hdfile)

if bdata:
    polygon = non_burn
    polygon_non_burn_file = os.path.join(datasets, polygon + ".geojson")
    non_burn_subarray, non_burn_transform, non_burn_mask = create_subset(polygon_non_burn_file, non_burn_subarray, ofiles, hdfile)
    non_burn_df = df
    
def save_image():
    dst_filename = os.path.join(outfolder, "%s_%s_%s_%s_%s.tif"%(polygon.split("_")[0], polygon.split("_")[3], date, pol_text, "burn-backscatter"))
    if not os.path.exists(outfolder):
        os.mkdir(outfolder)
    if os.path.exists(dst_filename):
        return
    driver = gdal.GetDriverByName( 'GTiff' )
    dst_ds=driver.Create(dst_filename, curr_arr.shape[1],curr_arr.shape[0], 1, gdal.GDT_Float32) # is shape the wrong way around?
    dst_ds.SetGeoTransform(new_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(27700)
    dest_wkt = srs.ExportToWkt()
    dst_ds.SetProjection (dest_wkt) # is this right?

    dst_ds.GetRasterBand(1).WriteArray( curr_arr )
    print("Saved to: {}".format(dst_filename))


@interact(img_num=(1,len(burn_subarray),1), polarisation=['VV','VH'])
def plot_polygons(polarisation='VV', img_num=0):
    global date
    global pol_text
    global curr_arr
    pol_text = polarisation
    pol = polar_dict[polarisation]
    date = list_of_dates[img_num-1]
    curr_arr = np.copy(burn_subarray[(img_num-1), pol, :, :])
    curr_arr[mask==0] = np.nan # set anything outside the polygon to np.nan
    fig = plt.figure(figsize=((20,10)))
    ax = fig.add_subplot(121)
    ax.imshow(curr_arr)
    plt.title("Image Date: %s"%(date))
    
    curr_arr2= np.copy(burn_subarray[(img_num-1), pol, :, ])
    ax2 = fig.add_subplot(122)
    ax2.imshow(curr_arr2)
    ax2.contour(mask > 0, levels=[2], colors=['r'])
    
    plt.show()
    
savebutton = interactive(save_image, {'manual' : True, 'manual_name' : 'Save'})

display(savebutton)

def save_image2():
    dst_filename = os.path.join(outfolder, "%s_%s_%s_%s_%s.tif"%(polygon.split("_")[0], polygon.split("_")[3], date2, pol_text, "non-burn-backscatter"))
    if not os.path.exists(outfolder):
        os.mkdir(outfolder)
    if os.path.exists(dst_filename):
        return
    driver = gdal.GetDriverByName( 'GTiff' )
    dst_ds=driver.Create(dst_filename, curr_arr3.shape[1],curr_arr3.shape[0], 1, gdal.GDT_Float32) # is shape the wrong way around?
    dst_ds.SetGeoTransform(non_burn_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(27700)
    dest_wkt = srs.ExportToWkt()
    dst_ds.SetProjection (dest_wkt) # is this right?

    dst_ds.GetRasterBand(1).WriteArray( curr_arr3 )
    print("Saved to: {}".format(dst_filename))


@interact(img_num_non_burn=(1,len(non_burn_subarray),1), polarisation_non_burn=['VV','VH'])
def plot_polygons2(polarisation_non_burn='VV', img_num_non_burn=0):
    global date2
    global pol_text
    global curr_arr3
    pol_text = polarisation_non_burn
    pol = polar_dict[polarisation_non_burn]
    date2 = list_of_dates[img_num_non_burn-1]
    curr_arr3 = np.copy(non_burn_subarray[(img_num_non_burn-1), pol, :, :])
    curr_arr3[non_burn_mask==0] = np.nan # set anything outside the polygon to np.nan
    fig2 = plt.figure(figsize=((20,10)))
    ax3 = fig2.add_subplot(121)
    ax3.imshow(curr_arr3)
    plt.title("Image Date: %s"%(date))
    
    curr_arr4= np.copy(non_burn_subarray[(img_num_non_burn-1), pol, :, ])
    ax4 = fig2.add_subplot(122)
    ax4.imshow(curr_arr4)
    ax4.contour(non_burn_mask > 0, levels=[2], colors=['r'])
    
    plt.show()
    
savebutton2 = interactive(save_image2, {'manual' : True, 'manual_name' : 'Save'})
display(savebutton2)

interactive(children=(Dropdown(description='polarisation', options=('VV', 'VH'), value='VV'), IntSlider(value=…

interactive(children=(Button(description='Save', style=ButtonStyle()), Output()), _dom_classes=('widget-intera…